#### The content of this file is based on: https://github.com/pescap/HPOMax/blob/main/HPO-PINNs/Dirichlet/HPO-2D.ipynb


In [ ]:
#import sys
#sys.path.insert(0,'/root/shared/nans/deepxde')

import skopt
import seaborn as sns
from model_gp import fitness
import pandas as pd
from matplotlib import pyplot as plt 
import numpy as np
from matplotlib.colors import LogNorm, Normalize
import seaborn as sns

In [ ]:
# Cost as function of depth and length
def formula(N , L):
    return N * (2+ 1) + (L-2) * N * (N + 1) + (N+1) 


def parse_results(name):
    search_result = skopt.load(name + '.pkl')       # load search_result
    n_calls = search_result.func_vals.shape[0]      # get n_calls
    test = []
    metric = []
    train = []
    texec = []

    # append results from all configs
    for i in range(n_calls):
        temp_name = name + '-'  + str(i) 
        train.append(skopt.load(temp_name + 'train.pkl'))
        test.append(skopt.load(temp_name + 'test.pkl')) 
        metric.append(skopt.load(temp_name + 'metric.pkl'))
        texec.append(skopt.load(temp_name + 'texec.pkl'))      

    # allocate
    min_train = np.zeros(n_calls)
    min_test = np.zeros(n_calls)
    min_metric = np.zeros(n_calls)

    # Find mest values for each config
    for i in range(n_calls):
        imin = test[i].argmin()
        min_train[i] = train[i][imin]
        min_test[i] = test[i][imin]
        min_metric[i] = metric[i][imin]

    # create dataframe of best values
    df1 = pd.DataFrame(np.column_stack([min_train, min_test, min_metric, texec]), columns = ['train', 'test', 'metric', 'texec'])

    # Remove nan in metric
    max_train = df1[~np.isnan(df1.train)].max()
    max_test =  df1[~np.isnan(df1.test)].max()
    max_metric = df1[~np.isnan(df1.metric)].max()
    
    # Set nan values to max
    df1[np.isnan(df1.train)] = max_train
    df1[np.isnan(df1.test)] = max_test
    df1[np.isnan(df1.metric)] = max_metric


    # Create dataframe of all configs
    df0 = pd.DataFrame(search_result.x_iters, columns=['lr', 'd', 'w', 'a']) 
    df0['f'] = search_result.func_vals.astype(float)        # function values for each iteration
    df0['global_index'] = np.arange(df0.shape[0])           # Configuration number

    # Information: learning rate, depth, width, activation
    df0['information'] = '[' +df0.lr.map(lambda x : "{:.2e}".format(x)).astype(str)+ ',' + df0.d.astype(str) + ','+ df0.w.astype(str)+ ',' + df0.a.astype(str) + ']'
    df0['cost'] = formula(df0.w, df0.d + 1)     # cost:  ???
    
    # Concatenate data frames
    df2 = pd.concat([df0, df1], axis = 1)
    
    # allocate
    conv = np.zeros(n_calls)
    conv_metric = np.zeros(n_calls)
    
    # Conv, conv_metric columns
    for i in range(n_calls):
        conv[i] = df2.f.values[:i + 1].min()
        conv_metric[i] = df2.metric.values[:i + 1].min()
    df2['conv'] = conv
    df2['conv_metric'] = conv_metric
    
    # if f == 100k we just take the max value
    df2.loc[df2.f == 100000, 'f'] = df2.f[df2.f != 100000].max()
    #df2.loc[df2.f == 100000, 'train'] = df2.train[df2.f != 100000].max()
    #df2.loc[df2.f == 100000, 'test'] = df2.test[df2.f != 100000].max()
    
    # sort by function values
    df = df2.sort_values(['f'], ascending = [True])

    # Convert types and format
    df['cost'] = df['cost'].astype(float)
    df[['global_index','f']] = df[['global_index','f']].astype(float)
    df['f'] = df['f'].astype(float)
    df['f_format'] = df.f.map(lambda x : "{:.2e}".format(x)).astype(str)
    df['cost_format'] = df.cost.map(lambda x : "{:.2e}".format(x)).astype(str)
    
    return df, train, test, metric





In [ ]:
name = 'results/burgersgp'

n_calls = 11
df, trainF, testF, metricF = parse_results(name)

# new columns
df['local_index'] = (np.arange(n_calls) + 1).astype(int)    # index here
df['log10f'] = np.log10(df.f)                               # log10 transform of f
df['log10metric'] = np.log10(df.metric)                     # log10 transform of metric
df['log10train'] = np.log10(df.train)                       # log10 transform of train
df['log10test'] = np.log10(df.test)                         # log10 transform of test
df['log10conv'] = np.log10(df.conv)                         # log10 transform of conv
df['log10conv_metric'] = np.log10(df.conv_metric)           # log10 transform of conv_metric

search_result = skopt.load(name + '.pkl')                   # load search_result

df

In [ ]:
melt1 = df.melt(id_vars =['local_index'], value_vars =  ['log10train', 'log10test'])
melt1

In [ ]:
melt1 = df.melt(id_vars =['global_index'], value_vars =  ['log10conv','log10conv_metric'])
melt1

In [ ]:
sns.set_style("ticks",{'axes.grid' : True})

# 1 row with 2 plots
fig, (ax1, ax2) = plt.subplots(1,2 , sharey=True, figsize= (12, 5))

g = sns.pointplot(data = melt1, 
                 x = 'local_index',                                   
                 palette = sns.color_palette('rocket_r', n_colors=3), # colors
                 y = 'value',
                 scale = .8,#, s = 80, lw = 2,
                 hue='variable',
                 ax = ax1) # plug into ax1
                 
#ax.set_xlabel(r'$\lambda_{M-m}^+$ Ranking')
#ax1.set_title(r'$\log_{10}\left(\mathcal{L}^\cdot_{\theta_K^+}[\lambda_{M-m}^+]\right)$', fontsize=16)

# title
ax1.set_title(r'$\log_{10}($loss$^\cdot[\lambda])$', fontsize=16)

# labels
ax1.set_ylabel(r'')
ax1.set_xlabel(r'Ranking', fontsize=16)

ax1.set_xlim(-1,12)                 # axis length
xticks = np.array([0, 3, 6, 9])     # ticks on x-axis
ax1.grid()                          # grid
labels = ax1.get_xticklabels()      # get x labels

ax1.set_xticks(xticks)
a = (xticks + 1).astype(str)
b = ['#' + s for s in list(a)]
ax1.set_xticklabels(b, fontsize= 14)
#ax1.set_yticklabels(labelsy, fontsize= 14)
ax1.xaxis.grid(color='gray', linestyle='dashed')
ax1.yaxis.grid(color='gray', linestyle='dashed')
#ax.tick_params(axis='both', which='major')
#title = ax.get_title()


handles, labels = ax1.get_legend_handles_labels() 

labels = ['train', 'test', 'metric']
legend = ax1.legend(loc = 'lower right', handles = handles, labels = labels, title = 'Plot:', fontsize=14)  
legend.get_title().set_fontsize('14')

# AX2

g = sns.pointplot(data = melt2, x = 'global_index', 
                 palette = sns.color_palette('rocket_r', n_colors=3)[1:3], 
                 y = 'value',
                 scale = .8,#, s = 80, lw = 2,
                 hue='variable',
                 ax = ax2)
ax2.set_xlabel(r'Iteration $m$', fontsize=16)

ax2.set_title(r'$\log_{10}($loss$^\cdot[\lambda^+_m])$', fontsize=16)
#ax2.set_title(r'$\log_{10}\left(\mathcal{L}^\cdot_{\theta_K^+}[\lambda_{m}^+]\right)$', fontsize=16)
ax2.set_ylabel(r'')

xticks = np.arange(0,10,2)

ax2.grid()
ax2.set_xlim(-1,12)

labels = ax2.get_xticklabels() # get x labels
labelsy = ax1.get_yticklabels() # get x labels

ax1.tick_params(axis='both', which='major', labelsize=14)


ax2.set_xticks(xticks)
ax2.set_xticklabels((xticks).astype(str), fontsize=14)

ax2.xaxis.grid(color='gray', linestyle='dashed')
ax2.yaxis.grid(color='gray', linestyle='dashed')
#title = ax.get_title()


handles, labels = ax2.get_legend_handles_labels() 

labels = ['test', 'metric']
legend = ax2.legend(loc = 'upper right', handles = handles, labels = labels, title = 'Plot:',fontsize =14)  
legend.get_title().set_fontsize('14')


plt.subplots_adjust(wspace=0.075, hspace=0.075)

plt.savefig('plots/s01.pdf')

# Partial dependence plot

In [ ]:
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
#from matplotlib import pyplot as plt

fig = plt.figure()
test = plot_objective(search_result, plot_dims = ['learning_rate', 'num_dense_nodes', 'num_dense_layers', 'activation'], show_points=True, size =3.8, cmap = 'rocket_r')

plt.savefig('plots/s03.pdf', bbox_inches='tight')

# Parameter plots

In [ ]:
fmin, fmax = df.f.min(), df.f.max()
cmin, cmax = df.cost.min(), df.cost.max()

plt.rc('font', family='serif')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (6,17), gridspec_kw={'width_ratios': [1,1.5]})
g =  sns.heatmap(df[['train']], annot =df[['f_format']], fmt = '' , cmap = 'rocket_r', ax = ax1, cbar_kws = dict(use_gridspec=False,location="left"), cbar = None, xticklabels = [r'$\mathcal{L}(\theta_K^\star, \lambda_n)$'])
g =  sns.heatmap(df[['cost']], annot =df[['information']], fmt = '' , cmap = 'rocket_r', ax = ax2, cbar = None, xticklabels = [r'$|\Theta|$'])
ax1.xaxis.tick_bottom() # x axis on top

#ax1.xaxis.set_label_position('top')


ax2.xaxis.tick_bottom() # x axis on top

plt.savefig('plots/s04.pdf')



In [ ]:
# Generate the colorbar

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize = (15,8))

g =  sns.heatmap(df[['f']], annot =df[['f_format']], fmt = '' , cmap = 'rocket_r', ax = ax1, cbar_kws = dict(use_gridspec=False, orientation = "horizontal", format = '%.2e'), cbar = True, xticklabels = [r'$\mathcal{L}(\theta_K^\star, \lambda_n)$'], cbar_ax =ax2)
g =  sns.heatmap(df[['cost']], annot =df[['information']], fmt = '' , cmap = 'rocket_r', ax = ax3, cbar = True, xticklabels = [r'$|\Theta|$'], cbar_kws = dict(use_gridspec=False, orientation = "horizontal", format = '%.2e'), cbar_ax=ax4)

#plt.savefig('plots/s05.pdf')

In [ ]:
print('fmin-fmax', fmin, fmax)
print('cmin-cmax', cmin, cmax)

# Convergence of best model

In [ ]:

index = 0
dfbest = pd.DataFrame(np.column_stack([np.arange(trainF[index].shape[0]) * 1000, np.log10(trainF[index]), np.log10(testF[index]),np.log10(metricF[index])]), columns = ['epoch', 'train', 'test', 'metric'])    

melt = dfbest.melt(id_vars = ['epoch'], value_vars = ['train', 'test', 'metric'])


sns.set_style("ticks",{'axes.grid' : True})

fig, ax = plt.subplots(figsize= (8,5))

g = sns.pointplot(data = melt, x = 'epoch', 
                 palette = sns.color_palette('rocket_r', n_colors=3), 
                 y = 'value',
                 scale = .8,# s = 80, lw = 2,
                 hue='variable',
                 ax = ax)
                 
                 
ax.set_xlabel(r'epoch $k$', fontsize=  16)
ax.set_title(r'$\log_{10}( $loss$^\cdot[\lambda_{97}])$', fontsize=16)

#ax2.set_title(r'$\log_{10}($loss$^\cdot[\lambda^+_m])$', fontsize=16)

#xticks = np.array([0, 10000, 20000, 30000, 40000, 50000])#+1

xticks = np.array([0, 10, 20, 30, 40, 50])
xlabels = ['0', '10,000', '20,000', '30,000', '40,000', '50,000']
ax.set_xlim(-2,52)

ax.grid()

#labels = ax.get_xticklabels() # get x labels
ax.set_ylabel(r'', fontsize=  16)

ax.set_xticks(xticks)
ax.set_xticklabels(xlabels)
ax.xaxis.grid(color='gray', linestyle='dashed')
ax.yaxis.grid(color='gray', linestyle='dashed')
#ax.tick_params(axis='both', which='major')
title = ax.get_title()


handles, labels = ax.get_legend_handles_labels() 

labels = ['train', 'test', 'metric']
legend = ax.legend(loc = 'upper right', handles = handles, labels = labels, title = 'Plot', fontsize =14)
legend.get_title().set_fontsize('14')
ax.tick_params(axis='both', which='major', labelsize=14)

plt.savefig('plots/s12.pdf', bbox_inches='tight')